In [1]:
%matplotlib notebook

In [232]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndimage
import time
from tqdm import tqdm

In [ ]:
class DiffMatrix():
    
    def __init__(self,
                size= 300,
                seed = 42,
                diffusion_characteristics = 'default'):
#       Reaction caracteristics
        if diffusion_characteristics == 'default':
            self.diff_a = 1
            self.diff_b = 0.5
            self.feed_rate = 0.055
            self.kill_rate = 0.062
            self.timestep = 1
            self.diff_func = self.laplacian2D

            self.size = size
            self.matrix_shape = (int(self.size), int(self.size))
            self.seed= seed
            self.matrix_a = self.initiate_matrix()
            self.matrix_b = self.initiate_matrix()
            self.matrix_a_new = self.initiate_matrix()
            self.matrix_b_new = self.initiate_matrix()
       
    def fill_matrix_tiny(self):
        self.matrix_a[1][2] = 0.3
        self.matrix_a[1][3] = 0.3
        self.matrix_a[2][2] = 0.3

    def fill_matrix_random(self):
        self.matrix_a = np.random.rand(*self.matrix_shape)
        self.matrix_b = np.random.rand(*self.matrix_shape)

        
    def initiate_matrix(self):
        matrix = np.zeros(self.matrix_shape)
        return matrix
        
            
    def diffuse(self):
        # diffuse trough the space
        self.diffuse_a = self.matrix_a * self.laplacian2D(self.matrix_a) * self.diff_a
        self.diffuse_b = self.matrix_b * self.laplacian2D(self.matrix_b) * self.diff_b
        
    def react(self):
        #consume a and turn it into b
        self.react_a = self.matrix_a * self.matrix_b * self.matrix_b
        self.react_b = self.matrix_a * self.matrix_b * self.matrix_b
        
    def feed(self):
        # create a up until 1
        self.feed_a = self.feed_rate * (1- self.matrix_a)
        #bestroy b if any
        self.feed_b = (self.feed_rate + self.kill_rate)* self.matrix_b

    
    def find_multiplier(self,x,y):
        multi_dict = {
            0: -1,
            1: 0.2,
            2: 0.05
        }
        total = abs(x) + abs(y)
        return multi_dict[total]
    
        
    def laplacian2D(self, matrix):
        x_coord = (-1,0,1)
        y_coord = (-1,0,1)
        matrix_new = np.empty(self.matrix_shape)
        
        for x in x_coord:
            rolled_x = np.roll(matrix,x,axis=0)
            for y in y_coord:
                multi = self.find_multiplier(x,y)
                rolled_y = np.roll(rolled_x,y,axis=1)
                matrix_new += (multi * rolled_y)
#                 print(f'x:{x} y:{y}')
#                 print(multi * rolled_y)
#                 print(matrix_new)
        return matrix_new
    
     
    def lap_filter(self,x):
        return x.max()
#         import pdb; pdb.set_trace()
    dm.matrix_a = ndimage.generic_filter(dm.matrix_a, self.lap_filter, size=(3,3))

        
    def _next(self):
        print('=== start')
        print(self.matrix_a)
        print('--')
        print(self.matrix_b)
        print('diff')
        self.diffuse()
        print(self.diffuse_a)
        print('--')
        print(self.diffuse_b)
        print('react')
        self.react()
        print(self.react_a)
        print('--')
        print(self.react_b)
        print('feed')
        self.feed()
        print(self.feed_a)
        print('--')
        print(self.feed_b)
        print('update')
        
        
        dm.matrix_a = ndimage.generic_filter(dm.matrix_a, self.lap_filter, size=(3,3))
        
        
        self.matrix_a += self.diffuse_a - self.react_a + self.feed_a
        self.matrix_b += self.diffuse_b + self.react_b - self.feed_b


In [247]:
dm = DiffMatrix(size = 7)
# dm.fill_matrix_random()
dm.fill_matrix_tiny()
print(dm.matrix_a)
print(dm.matrix_b)

[[0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.3 0.3 0.  0.  0. ]
 [0.  0.  0.3 0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0. ]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


In [ ]:
for i in range(1):
    dm._next()
print(dm.matrix_a)
print(dm.matrix_b)

=== start
[[0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.3 0.3 0.  0.  0. ]
 [0.  0.  0.3 0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0. ]]
--
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
diff
[[ 0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.     -0.054  -0.0675  0.      0.      0.    ]
 [ 0.      0.     -0.0675  0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.    ]]
--
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -0. -0.  0.  0.  0.]
 [ 0.  0. -0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  

In [246]:
# plt.imshow(dm.matrix_a, cmap='binary', interpolation='nearest')

In [241]:
def test(x):
    return x.mean()

dm.matrix_a = ndimage.generic_filter(dm.matrix_a, test, size=(3,3))

In [242]:
result

array([[0.        , 0.00159573, 0.00301439, 0.00301439, 0.00141866,
        0.        , 0.        ],
       [0.        , 0.00301439, 0.00443306, 0.00443306, 0.00141866,
        0.        , 0.        ],
       [0.        , 0.00301439, 0.00443306, 0.00443306, 0.00141866,
        0.        , 0.        ],
       [0.        , 0.00141866, 0.00141866, 0.00141866, 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])

In [205]:
plt.imshow(a, cmap='binary', interpolation='nearest')
    plt.show()

5